In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import spacy
import dataset
import analyze

In [2]:
likes_path = "Dataset/easytour-likes-2022-03MAR-21.json"
schede_path = "Dataset/easytour-schede-2022-03MAR-21.json"
data_path = "Dataset/data.csv"

In [3]:
f_schede = open(schede_path)
data_schede = json.load(f_schede)
print("Fields: {}".format(data_schede[0].keys())) 
print("Number of samples: {}".format(len(data_schede)))

Fields: dict_keys(['id', 'userId', 'userName', 'documentType', 'title', 'description', 'locations', 'feature', 'duration', 'imageId', 'previewId', 'distance', 'datePublishing', 'widgets', 'counterUseful', 'tags', 'numberOfPositions', 'creationDate', 'videoId', 'version', 'categories', 'media', 'shareType', 'status', 'lastUpdate', 'versionHistory', 'viewCounter', 'device'])
Number of samples: 4356


In [4]:
#print("First sample: {}".format(data_schede[8]))

In [5]:
f_likes = open(likes_path)
data_likes = json.load(f_likes)
print("Number of samples: {}".format(len(data_likes)))
print("Fields: {}".format(data_likes[0].keys()))
print("First sample: {}".format(data_likes[0]))

Number of samples: 1503
Fields: dict_keys(['id', 'userId', 'userName', 'lastUpdate'])
First sample: {'id': '622c3b8ac992ca28f33c7f21', 'userId': 606, 'userName': 'Dale On Travel', 'lastUpdate': '2022-03-19T07:56:20.868Z'}


In [6]:
df_schede = pd.read_json(schede_path)
df_likes = pd.read_json(likes_path)

<h3>Data Analysis</h3>

In [7]:
df_schede.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4356 entries, 0 to 4355
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 4356 non-null   object             
 1   userId             4356 non-null   int64              
 2   userName           4356 non-null   object             
 3   documentType       4356 non-null   object             
 4   title              4356 non-null   object             
 5   description        4355 non-null   object             
 6   locations          4356 non-null   object             
 7   feature            4356 non-null   object             
 8   duration           4355 non-null   float64            
 9   imageId            4356 non-null   object             
 10  previewId          1763 non-null   object             
 11  distance           4356 non-null   float64            
 12  datePublishing     4345 non-null   object       

In [8]:
df_schede.describe()

,userId,duration,distance,counterUseful,numberOfPositions,version,viewCounter
count,4356.000000,4.355000e+03,4356.000000,4356.000000,4356.000000,4355.0000,4355.000000
mean,394.564968,1.646045e+07,1.862263,0.366162,1.184803,5.5031,40.477153
std,147.424098,5.898965e+07,122.909338,0.734108,0.789168,7.8346,30.050625
min,77.000000,0.000000e+00,0.000000,0.000000,1.000000,2.0000,0.000000
25%,236.000000,3.600000e+06,0.000000,0.000000,1.000000,2.0000,32.000000
50%,435.000000,3.600000e+06,0.000000,0.000000,1.000000,4.0000,43.000000
75%,529.000000,1.080000e+07,0.000000,1.000000,1.000000,6.0000,50.000000
max,642.000000,1.296000e+09,8112.016276,8.000000,12.000000,204.0000,694.000000


In [9]:
df_schede_fields = df_schede[["id","userId", "title", "description", "duration", "datePublishing", "creationDate", "counterUseful", "categories", "viewCounter"]]
df_likes_ids = df_likes[["id", "userId"]]

In [10]:
df_schede_fields

,id,userId,title,description,duration,datePublishing,creationDate,counterUseful,categories,viewCounter
0,62355f98272ae5672fbc3ac7,632,Spiagge selvagge a Phuket 🇹🇭,"Phuket, il paradiso per gli amanti del sole, d...",0.0,2022-03-19T04:44:09.503Z,2022-03-19T04:44:08.000+00:00,0,[Viaggi],2.0
1,620abb33ff8ae67f37cafe3c,442,Freedom beach,Forse la spiaggia più bella di tutta l’isola d...,3600000.0,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,0,[Viaggi],413.0
2,620aa536ff8ae67f37cafb06,442,Accarezzare le tigri,So che dietro queste cose c’è spesso del marci...,3600000.0,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,0,[Viaggi],414.0
3,62349ff2272ae5672fbc3135,632,"Big Buddha, Phuket 🇹🇭",Phuket è conosciuta per essere la meta predile...,0.0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,0,[Viaggi],4.0
4,6234aa65272ae5672fbc34c2,632,"Wat Chalong, il tempio più venerato di Phuket 🇹🇭","Phuket non è solo spiagge, ci sono anche altri...",0.0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,0,[Viaggi],4.0
...,...,...,...,...,...,...,...,...,...,...
4351,61f57f62306802033034ceed,536,Avventura in Ecuador - le cascate di Baños,Ripartiamo dopo la nostra esperienza in Amazzo...,259200000.0,2022-01-29T17:54:42.761Z,2022-01-29T17:54:42.000+00:00,0,[Viaggi],42.0
4352,61f668fa306802033034d239,536,Avventura in Ecuador - el Volcán Chimborazo,E così ci lasciamo alle spalle anche la splend...,57600000.0,2022-01-30T10:31:22.977Z,2022-01-30T10:31:22.000+00:00,0,[Viaggi],35.0
4353,620bbf26b1675d6cc11232f8,431,"Lomas de Paraíso VMT, polmone verde","La chiamano ""ciudad gris"" Lima. Per circa quat...",9000000.0,2022-02-15T14:56:39.554Z,2022-02-15T14:56:38.000+00:00,1,[Viaggi],37.0
4354,61f3c6add91e9928f6e0922a,431,Il meglio di Lima,È quasi impossibile trovare un angolo del Perù...,172800000.0,2022-02-01T16:57:01.557Z,2022-01-28T10:34:21.000+00:00,1,[Viaggi],51.0


<h3>Preprocessing</h3>

<h5>Null or empty values</h5>

In [11]:
#Check if there are null fields
df_schede_fields.isnull().sum()

id                  0
userId              0
title               0
description         1
duration            1
datePublishing     11
creationDate        0
counterUseful       0
categories        138
viewCounter         1
dtype: int64

In [12]:
#empty description -> remove or use '' ?
df_schede_fields["description"].fillna('', inplace=True)
#if duration is null -> set to 0
df_schede_fields["duration"].fillna(0, inplace=True)
#if datePublishing is null -> set it equal to creationDate
df_schede_fields["datePublishing"].fillna(df_schede_fields.creationDate, inplace=True)
#if viewCounter is null -> set to 0
df_schede_fields["viewCounter"].fillna(0, inplace=True)

/tmp/ipykernel_8658/1831612475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schede_fields["description"].fillna('', inplace=True)
/tmp/ipykernel_8658/1831612475.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schede_fields["duration"].fillna(0, inplace=True)
/tmp/ipykernel_8658/1831612475.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schede_fields["datePublishing"].fillna(df_schede_fields.creationDate, inplace

<h5>Categories into one hot encoding</h5>

In [13]:
categories = df_schede_fields[df_schede_fields["categories"].isnull() == False]["categories"].apply(list)
categories_list = list({x for l in categories for x in l})
categories_list

['Cultura',
 'Aria aperta',
 'Aria Aperta',
 'Food',
 'Sport',
 'sport',
 'Tempo libero',
 'Viaggi']

In [14]:
def lower_list(l):
    if type(l)!=list:
        return []
    res = [x.lower() for x in l]
    return res

one_hot_enc_categories = df_schede_fields["categories"].apply(lambda x: lower_list(x))

In [15]:
df_schede_fields["categories"] = one_hot_enc_categories

/tmp/ipykernel_8658/2114275071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schede_fields["categories"] = one_hot_enc_categories


In [16]:
mlb = MultiLabelBinarizer()

schede_with_categories = df_schede_fields.join(
            pd.DataFrame(
                mlb.fit_transform(df_schede_fields.pop('categories')),
                index=df_schede_fields.index,
                columns=mlb.classes_))

In [17]:
schede_with_categories

,id,userId,title,description,duration,datePublishing,creationDate,counterUseful,viewCounter,aria aperta,cultura,food,sport,tempo libero,viaggi
0,62355f98272ae5672fbc3ac7,632,Spiagge selvagge a Phuket 🇹🇭,"Phuket, il paradiso per gli amanti del sole, d...",0.0,2022-03-19T04:44:09.503Z,2022-03-19T04:44:08.000+00:00,0,2.0,0,0,0,0,0,1
1,620abb33ff8ae67f37cafe3c,442,Freedom beach,Forse la spiaggia più bella di tutta l’isola d...,3600000.0,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,0,413.0,0,0,0,0,0,1
2,620aa536ff8ae67f37cafb06,442,Accarezzare le tigri,So che dietro queste cose c’è spesso del marci...,3600000.0,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,0,414.0,0,0,0,0,0,1
3,62349ff2272ae5672fbc3135,632,"Big Buddha, Phuket 🇹🇭",Phuket è conosciuta per essere la meta predile...,0.0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,0,4.0,0,0,0,0,0,1
4,6234aa65272ae5672fbc34c2,632,"Wat Chalong, il tempio più venerato di Phuket 🇹🇭","Phuket non è solo spiagge, ci sono anche altri...",0.0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,0,4.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,61f57f62306802033034ceed,536,Avventura in Ecuador - le cascate di Baños,Ripartiamo dopo la nostra esperienza in Amazzo...,259200000.0,2022-01-29T17:54:42.761Z,2022-01-29T17:54:42.000+00:00,0,42.0,0,0,0,0,0,1
4352,61f668fa306802033034d239,536,Avventura in Ecuador - el Volcán Chimborazo,E così ci lasciamo alle spalle anche la splend...,57600000.0,2022-01-30T10:31:22.977Z,2022-01-30T10:31:22.000+00:00,0,35.0,0,0,0,0,0,1
4353,620bbf26b1675d6cc11232f8,431,"Lomas de Paraíso VMT, polmone verde","La chiamano ""ciudad gris"" Lima. Per circa quat...",9000000.0,2022-02-15T14:56:39.554Z,2022-02-15T14:56:38.000+00:00,1,37.0,0,0,0,0,0,1
4354,61f3c6add91e9928f6e0922a,431,Il meglio di Lima,È quasi impossibile trovare un angolo del Perù...,172800000.0,2022-02-01T16:57:01.557Z,2022-01-28T10:34:21.000+00:00,1,51.0,0,0,0,0,0,1


<h5>Text fields</h5>

In [18]:
schede_with_categories["title"] = schede_with_categories["title"].apply(lambda x: analyze.demojize(x))
schede_with_categories["description"] = schede_with_categories["description"].apply(lambda x: analyze.demojize(x))

In [19]:
nlp = spacy.load("it_core_news_lg")

def keep_just_nouns(field):
    doc = nlp(field.lower())
    return [token.lemma_ for token in doc if token.pos_=="NOUN"]

def remove_verbs_adj(field):
    doc = nlp(field.lower())
    return [token.lemma_ for token in doc if token.pos_ not in ["VERB", "ADJ", "ADV"]]

In [20]:
# Remove Verbs from description
schede_description = schede_with_categories.apply(
    lambda row: keep_just_nouns(row['description']),
    axis=1)

# Remove hastags, punctuation, stop words and numbers
schede_description = schede_description.apply(
    lambda row: dataset.clean_text_aslist(row))

In [21]:
# Remove Verbs from title
schede_title = schede_with_categories.apply(
    lambda row: keep_just_nouns(row['title']),
    axis=1)
    
# Remove hastags, punctuation, stop words and numbers
schede_title = schede_title.apply(
    lambda row: dataset.clean_text_aslist(row))

In [22]:
schede_description

0       [paradiso, amant, sole, mare, sole, cocktail, ...
1       [spiaggia, isola, spiaggia, mare, terra, fores...
2       [marcio, maltrattamento, animal, sconsiglio, p...
3       [meta, turista, assalto, spiaggia, acqua, mera...
4       [spiaggia, pena, tempio, big, tempio, tempio, ...
                              ...                        
4351    [esperienza, step, contatto, realtà, genitor, ...
4352    [spalla, esperienza, lascito, vista, catena, c...
4353    [mese, sole, capital, nuvola, nebbia, territor...
4354    [angolo, turista, calor, gent, bellezza, paesa...
4355    [viaggio, dubbio, isola, galapago, 50ina, isol...
Length: 4356, dtype: object

In [23]:
schede_with_categories["title"] = schede_title
schede_with_categories["description"] = schede_description

<h5>Dates</h5>

In [24]:
# Convert duration from ms to minutes
schede_with_categories['duration_min'] = schede_with_categories.apply(lambda row: row["duration"] / 60000, axis=1)

# Compute age of entry in days from today
schede_with_categories['days'] = 0
schede_with_categories.days = (pd.to_datetime("now").tz_localize('UTC') - pd.to_datetime(
    schede_with_categories["creationDate"])) // np.timedelta64(1, 'D')

/home/gianfree/.local/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [25]:
schede_with_categories

,id,userId,title,description,duration,datePublishing,creationDate,counterUseful,viewCounter,aria aperta,cultura,food,sport,tempo libero,viaggi,duration_min,days
0,62355f98272ae5672fbc3ac7,632,[spiaggia],"[paradiso, amant, sole, mare, sole, cocktail, ...",0.0,2022-03-19T04:44:09.503Z,2022-03-19T04:44:08.000+00:00,0,2.0,0,0,0,0,0,1,0.0,9
1,620abb33ff8ae67f37cafe3c,442,[],"[spiaggia, isola, spiaggia, mare, terra, fores...",3600000.0,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,0,413.0,0,0,0,0,0,1,60.0,41
2,620aa536ff8ae67f37cafb06,442,[tigr],"[marcio, maltrattamento, animal, sconsiglio, p...",3600000.0,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,0,414.0,0,0,0,0,0,1,60.0,41
3,62349ff2272ae5672fbc3135,632,[big],"[meta, turista, assalto, spiaggia, acqua, mera...",0.0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,0,4.0,0,0,0,0,0,1,0.0,10
4,6234aa65272ae5672fbc34c2,632,"[wat, tempio]","[spiaggia, pena, tempio, big, tempio, tempio, ...",0.0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,0,4.0,0,0,0,0,0,1,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,61f57f62306802033034ceed,536,"[avventura, cascata]","[esperienza, step, contatto, realtà, genitor, ...",259200000.0,2022-01-29T17:54:42.761Z,2022-01-29T17:54:42.000+00:00,0,42.0,0,0,0,0,0,1,4320.0,57
4352,61f668fa306802033034d239,536,[avventura],"[spalla, esperienza, lascito, vista, catena, c...",57600000.0,2022-01-30T10:31:22.977Z,2022-01-30T10:31:22.000+00:00,0,35.0,0,0,0,0,0,1,960.0,57
4353,620bbf26b1675d6cc11232f8,431,"[loma, polmon]","[mese, sole, capital, nuvola, nebbia, territor...",9000000.0,2022-02-15T14:56:39.554Z,2022-02-15T14:56:38.000+00:00,1,37.0,0,0,0,0,0,1,150.0,41
4354,61f3c6add91e9928f6e0922a,431,[lima],"[angolo, turista, calor, gent, bellezza, paesa...",172800000.0,2022-02-01T16:57:01.557Z,2022-01-28T10:34:21.000+00:00,1,51.0,0,0,0,0,0,1,2880.0,59


In [26]:
schede_with_categories.to_csv(data_path, index=False)

<h5>Tf-Idf</h5>

In [27]:
data = pd.read_csv(data_path)  

In [28]:
data

,id,userId,title,description,duration,datePublishing,creationDate,counterUseful,viewCounter,aria aperta,cultura,food,sport,tempo libero,viaggi,duration_min,days
0,62355f98272ae5672fbc3ac7,632,['spiaggia'],"['paradiso', 'amant', 'sole', 'mare', 'sole', ...",0.0,2022-03-19T04:44:09.503Z,2022-03-19T04:44:08.000+00:00,0,2.0,0,0,0,0,0,1,0.0,9
1,620abb33ff8ae67f37cafe3c,442,[],"['spiaggia', 'isola', 'spiaggia', 'mare', 'ter...",3600000.0,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,0,413.0,0,0,0,0,0,1,60.0,41
2,620aa536ff8ae67f37cafb06,442,['tigr'],"['marcio', 'maltrattamento', 'animal', 'sconsi...",3600000.0,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,0,414.0,0,0,0,0,0,1,60.0,41
3,62349ff2272ae5672fbc3135,632,['big'],"['meta', 'turista', 'assalto', 'spiaggia', 'ac...",0.0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,0,4.0,0,0,0,0,0,1,0.0,10
4,6234aa65272ae5672fbc34c2,632,"['wat', 'tempio']","['spiaggia', 'pena', 'tempio', 'big', 'tempio'...",0.0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,0,4.0,0,0,0,0,0,1,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,61f57f62306802033034ceed,536,"['avventura', 'cascata']","['esperienza', 'step', 'contatto', 'realtà', '...",259200000.0,2022-01-29T17:54:42.761Z,2022-01-29T17:54:42.000+00:00,0,42.0,0,0,0,0,0,1,4320.0,57
4352,61f668fa306802033034d239,536,['avventura'],"['spalla', 'esperienza', 'lascito', 'vista', '...",57600000.0,2022-01-30T10:31:22.977Z,2022-01-30T10:31:22.000+00:00,0,35.0,0,0,0,0,0,1,960.0,57
4353,620bbf26b1675d6cc11232f8,431,"['loma', 'polmon']","['mese', 'sole', 'capital', 'nuvola', 'nebbia'...",9000000.0,2022-02-15T14:56:39.554Z,2022-02-15T14:56:38.000+00:00,1,37.0,0,0,0,0,0,1,150.0,41
4354,61f3c6add91e9928f6e0922a,431,['lima'],"['angolo', 'turista', 'calor', 'gent', 'bellez...",172800000.0,2022-02-01T16:57:01.557Z,2022-01-28T10:34:21.000+00:00,1,51.0,0,0,0,0,0,1,2880.0,59


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False)
# description and title are put together
text = data['description'] + data['title']
tfidf_schede = tfidf_vectorizer.fit_transform((text))
tfidf_schede

<4356x17749 sparse matrix of type '<class 'numpy.float64'>'
	with 203045 stored elements in Compressed Sparse Row format>

<h2>Recommender systems</h2>

<h5>Content-based using just tf-idf and similarity on posts that user liked</h5>

In [30]:
#list of user
users = pd.concat([df_likes_ids.userId, data.userId]).drop_duplicates()
#list of posts
posts = data.id
#list of users and posts they liked
users_likes = df_likes_ids.groupby("userId").id.apply(list)
users_likes

userId
114    [61447a3224c29e59fe4c06ab, 61824f5f21889d3548f...
119    [6172da4472b6f336088b932b, 6218ec9f29ef4a75805...
129    [615acf21eb3bf47494b8e64b, 61aa039788b2e606829...
149    [61faf5d953a6a72116188d5b, 61f41fe330680203303...
150    [61b72f34e364b82f2ac8c88c, 61f3c801d91e9928f6e...
                             ...                        
630    [62215a990a0ebd53f7469748, 62174fea99163528c06...
632                           [6229df13aeb26768dc408e81]
637    [6212993a4f9d177c90f3830f, 6224bdb835684f072b2...
639                           [620a39a4e7ce7d5992612316]
640    [622aeada99fd4c40442f3082, 6217ab40ae6735171d5...
Name: id, Length: 116, dtype: object

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_schede, tfidf_schede)

In [32]:
def get_recommendations(user_id: int):
    # Get the index of the user that we want to recommend posts to
    user_id_idx = df_schede[df_schede == user_id].index[0]
    # Get the pairwise similarity scores of all posts with that user
    sim_scores = list(enumerate(cosine_sim[user_id_idx]))
    # Sort the posts based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar posts
    sim_scores = sim_scores[1:11]
    # Get the blog post ids
    post_ids = [i[0] for i in sim_scores]
    # Return the blog posts
    return df_schede.iloc[post_ids]

def print_recommendations(recommended_posts):
    for i in range(len(recommended_posts)):
        print("Post #" + str(i+1) + ":")
        print("Post ID: " + str(recommended_posts.iloc[i].id))
        print("Title: " + str(recommended_posts.iloc[i].title))
        print("Description: " + str(recommended_posts.iloc[i].description))
        print("\n")

In [33]:
id = 632
#print("Posts that user {} liked:".format(id))
liked_posts = df_likes_ids[df_likes_ids["userId"] == id]
#print(liked_posts)
print("Posts:")
for _,p in liked_posts.iterrows():
    post = df_schede[df_schede["id"]== p["id"]]
    if not post.description.empty:
        print(post.title)
        print("-"*100)

Posts:
92    Bemo - immergiti nella vita locale indonesiana
Name: title, dtype: object
----------------------------------------------------------------------------------------------------


In [34]:
recommended_posts = get_recommendations(id)
print_recommendations(recommended_posts)

Post #1:
Post ID: 61f6ce0b306802033034e3e7
Title: Le spiagge più belle della Sardegna
Description: La Sardegna è una regione ricca di spiagge meravigliose e mare dal colore cristallino, cercheremo di aiutarvi a trovare le spiagge più belle e quelle che secondo noi non potete assolutamente perdervi se fate un viaggio in questa regione fantastica!

📌Berchida
Berchida è una delle spiagge più lunghe e spaziosi della Sardegna. La Spiaggia è posizionata poco sopra il Golfo di Orosei ed è quasi tutta di sabbia finissima, inoltre, riesce a sorprenderti per i colori della sua acqua, davvero cristallina. La grandezza della spiaggia gioca a favore anche per la possibilità di avere un po’ di privacy e relax.

📌Spiaggia di S’archittu
Questa spiaggia è situata nella parte ovest della Sardegna poco sopra Oristano. La spiaggia in sé non è molto grande quindi vi consigliamo di arrivare presto per trovare posto. La particolarità è la presenza di un arco, appunto Archittu, formato dalla scogliera che rac

<h5>Collaborative filtering</h5>